In [1]:
import json
import os

from tqdm import tqdm
import pandas as pd
import jams
import librosa
import madmom
import crema
import pumpp
import mir_eval

Using TensorFlow backend.


In [2]:
def ann_to_label(ann):
    intervals, labels = ann.to_interval_values()
    dur_tracker = dict()

    for interval, label in zip(intervals, labels):
        dur = interval[1] - interval[0]
        if label not in dur_tracker:
            dur_tracker[label] = dur
        else:
            dur_tracker[label] += dur

    # return the key with the max value as the gloabl key_mode
    return max(dur_tracker, key=dur_tracker.get)


class MadmomKeyAnalyzer(object):
    def __init__(self):
        self.key_processor = madmom.features.key.CNNKeyRecognitionProcessor()

    def to_label(self, filename):
        return madmom.features.key.key_prediction_to_label(self.key_processor(filename))
    
    def to_ann(self, filename):
        label = self.to_label(filename)
        label = ':'.join(label.split(' '))
        file_dur = librosa.get_duration(filename=filename)
        ann = jams.Annotation(namespace='key_mode', time=0, duration=file_dur)
        ann.append(time=0, duration=file_dur, value=label)
        return ann


class CremaKeyAnalyzer(object):
    def __init__(self, model_weights='model.h5'):
        self.model = crema.models.key.KeyModel(model_weights=model_weights)
    
    def to_ann(self, filename):
        return self.model.predict(filename=filename)
    
    def to_label(self, filename):
        ann = self.to_ann(filename)
        label = ann_to_label(ann)
        return ' '.join(label.split(':'))

In [3]:
mka = MadmomKeyAnalyzer()
cka = CremaKeyAnalyzer()

W0923 17:00:29.456259 4605605312 deprecation_wrapper.py:119] From /Users/tom/.pyenv/versions/miniconda3-latest/envs/keys/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 17:00:29.524981 4605605312 deprecation_wrapper.py:119] From /Users/tom/.pyenv/versions/miniconda3-latest/envs/keys/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0923 17:00:29.577305 4605605312 deprecation_wrapper.py:119] From /Users/tom/.pyenv/versions/miniconda3-latest/envs/keys/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0923 17:00:29.578106 4605605312 deprecation_wrapper.py:119] From /Users/tom/.pyenv/versions/miniconda3-latest/envs/keys/lib/python3.7/site-packages

In [6]:
f_name = librosa.util.example_audio_file()

m_label = mka.to_label(f_name)
c_label = cka.to_label(f_name)

In [7]:
m_label, c_label

('E major', 'E major')

## Next steps
- convert ground truth to label and to jams
- convert madmom label to jams and compare there
- *optional* convert crema jams to label and compare there

In [10]:
GS_HOME = "/Users/tom/Music/GuitarSet/"

gs_val_index = pd.read_json('dataset_indecies/gs_val_idx.json')[0]
gs_index = pd.read_json('dataset_indecies/guitarset_index.json')

In [16]:
def result_df_to_score_(result):
    t_score = 0
    m_score = 0
    c_score = 0

    for track_id, row in result.iterrows():
        t_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.truth)['Weighted Score']
        m_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.madmom)['Weighted Score']
        c_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.crema)['Weighted Score']

    print('weighted mirex scores:')
    print('\t madmom : {}'.format(m_score / t_score))
    print('\t crema  : {}'.format(c_score / t_score))

In [11]:
gs_index.keys()

Index(['audio_mic_comp', 'audio_mix_comp', 'jams_comp', 'audio_mic_solo',
       'audio_mix_solo', 'jams_solo'],
      dtype='object')

In [14]:
# ONLY ON COMP
label_true = []
label_madmom = []
label_crema = []

for val_id in tqdm(gs_val_index):
    jam_path = os.path.join(GS_HOME, gs_index.jams_comp[val_id])
    mic_path = os.path.join(GS_HOME, gs_index.audio_mic_comp[val_id])
    
    ann_true = jams.load(jam_path).search(namespace='key_mode')[0]
    l_true = ann_to_label(ann_true)
    label_true.append(' '.join(l_true.split(':')))
    label_madmom.append(mka.to_label(mic_path))
    label_crema.append(cka.to_label(mic_path))

gs_comp_result = pd.DataFrame(data={'truth':label_true, 'madmom':label_madmom, 'crema':label_crema}, 
                      index=gs_val_index)

result_df_to_score_(gs_comp_result)

weighted mirex scores:
	 madmom : 0.7000000000000001
	 crema  : 0.9805555555555555


In [15]:
# ONLY ON SOLO
label_true = []
label_madmom = []
label_crema = []

for val_id in tqdm(gs_val_index):
    jam_path = os.path.join(GS_HOME, gs_index.jams_comp[val_id])
    mic_path = os.path.join(GS_HOME, gs_index.audio_mic_solo[val_id])
    
    ann_true = jams.load(jam_path).search(namespace='key_mode')[0]
    l_true = ann_to_label(ann_true)
    label_true.append(' '.join(l_true.split(':')))
    label_madmom.append(mka.to_label(mic_path))
    label_crema.append(cka.to_label(mic_path))

gs_solo_result = pd.DataFrame(data={'truth':label_true, 'madmom':label_madmom, 'crema':label_crema}, 
                      index=gs_val_index)

result_df_to_score_(gs_solo_result)

100%|██████████| 36/36 [03:51<00:00,  6.12s/it]

weighted mirex scores:
	 madmom : 0.5444444444444446
	 crema  : 0.8166666666666668


## Next steps
- implement segment by segment key tag valuation.
- if it's not minor/major, then it's major.
- find more audio dataset to feed it
- do the MIDI dataset pipeline
- See how the structured output helps the model

In [20]:
AMAPS_HOME = '/Users/tom/Music/A-MAPS/'
val_index = pd.read_json('dataset_indecies/amaps_val_idx.json')[0]
amaps_index = pd.read_json('dataset_indecies/amaps_index.json')

In [21]:
amaps_index.index

Index(['MAPS_MUS-alb_esp2_AkPnCGdD', 'MAPS_MUS-alb_esp2_AkPnStgb',
       'MAPS_MUS-alb_esp2_SptkBGAm', 'MAPS_MUS-alb_esp2_SptkBGCl',
       'MAPS_MUS-alb_esp3_AkPnCGdD', 'MAPS_MUS-alb_esp4_AkPnStgb',
       'MAPS_MUS-alb_esp5_SptkBGCl', 'MAPS_MUS-alb_esp6_SptkBGCl',
       'MAPS_MUS-alb_se2_ENSTDkCl', 'MAPS_MUS-alb_se2_StbgTGd2',
       ...
       'MAPS_MUS-ty_november_ENSTDkAm', 'MAPS_MUS-ty_november_SptkBGCl',
       'MAPS_MUS-ty_oktober_SptkBGAm', 'MAPS_MUS-ty_september_AkPnStgb',
       'MAPS_MUS-ty_september_SptkBGAm', 'MAPS_MUS-waldstein_1_AkPnBsdf',
       'MAPS_MUS-waldstein_1_AkPnStgb', 'MAPS_MUS-waldstein_1_ENSTDkAm',
       'MAPS_MUS-waldstein_2_AkPnCGdD', 'MAPS_MUS-waldstein_3_SptkBGCl'],
      dtype='object', length=266)

In [25]:
label_true = []
label_madmom = []
label_crema = []

for track_id in tqdm(val_index):
    jam_path = os.path.join(AMAPS_HOME, amaps_index.jams[track_id])
    mic_path = os.path.join(AMAPS_HOME, amaps_index.audio[track_id])
    
    ann_true = jams.load(jam_path).search(namespace='key_mode')[0]
    l_true = ann_to_label(ann_true)
    label_true.append(' '.join(l_true.split(':')))
    label_madmom.append(mka.to_label(mic_path))
    label_crema.append(cka.to_label(mic_path))

amaps_result = pd.DataFrame(data={'truth':label_true, 'madmom':label_madmom, 'crema':label_crema}, 
                            index=val_index)

result_df_to_score_(amaps_result)

weighted mirex scores:
	 madmom : 0.6180000000000001
	 crema  : 0.638


In [26]:
amaps_result

,truth,madmom,crema
0,,,
MAPS_MUS-alb_se8_SptkBGAm,Eb major,Eb major,D# major
MAPS_MUS-chpn-p11_AkPnCGdD,B major,B major,B major
MAPS_MUS-chpn-p8_AkPnBcht,A major,F# minor,F# minor
MAPS_MUS-pathetique_3_ENSTDkAm,Eb major,C minor,D# major
MAPS_MUS-scn15_3_AkPnBcht,D major,G major,G major
MAPS_MUS-liz_et3_AkPnStgb,B major,G# minor,G# minor
MAPS_MUS-mz_333_2_AkPnCGdD,Eb major,Eb major,D# major
MAPS_MUS-mz_333_2_ENSTDkCl,Eb major,Eb major,D# major
MAPS_MUS-mz_333_2_StbgTGd2,Eb major,Eb major,D# major
